# **XGBRegressor (XGBoost)**

*Fonctionne avec Python 3.10.9 (Anaconda 23.3.1) et Keras 2.10.0*

## **Importation des bibliothèques**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pour les traîtements sur les variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modèle de ML utilisé
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
# Si on veut optimiser adam, notamment le learning rate
from keras.optimizers import Adam

# Pour gridsearch
from sklearn import model_selection

# Pour mesures des metrics des résultats
from sklearn import metrics

# Pour mise en forme des résultats
import colorama

# Désactiver un warning DeprecationWarning: KerasRegressor is deprecated
import warnings
warnings.filterwarnings('ignore')

## **Importation du dataset**

In [2]:
dataset_a_utiliser = "data_clean.xlsx"

data = pd.read_excel(dataset_a_utiliser)

## **Fonctions**

In [3]:
def split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification):

    ##############################
    # Split du dataset en X et y #
    ##############################

    global X
    global y

    X = data.loc[:,features_list] # On ajoute nos features dans le X
    y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y

    #logtransformer = FunctionTransformer(np.log, inverse_func = np.exp, check_inverse = True)
    #y = logtransformer.transform(y)

    ############################
    # Scaling et Encoding de X #
    ############################

    global feature_encoder

    # Ici RobustScaler() améliore un peu les résultats
    numeric_transformer = StandardScaler()

    categorical_transformer = OneHotEncoder()

    feature_encoder = ColumnTransformer(
                transformers=[
                    ('cat', categorical_transformer, categorical_features),    
                    ('num', numeric_transformer, numeric_features)
                    ]
                )
    X = feature_encoder.fit_transform(X)


    ####################################
    # Split de X et y en train et test #
    ####################################

    global X_train
    global X_test
    global y_train
    global y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=0,
                                                        stratify=eval(stratification))

## **Features**

In [4]:
######################
# Choix des features #
######################

dataset_name = data

# Si rien mettre []
categorical_features = [
                        'BuildingType', 
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'ZipCode',
                        #'YearBuilt' # Meilleurs résultats sans
                       ]

# Si rien mettre []
numeric_features = [
                    'NumberofBuildings',
                    'NumberofFloors',
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)',
                    'Latitude',
                    'Longitude',
                    #'age_bat' # meilleur score sans
                   ]

# Toutes les features
features_list = categorical_features + numeric_features

## ***Keras***



In [5]:
# Targets à estimer
targets = ['SiteEnergyUse_kBtu', 'TotalGHGEmissions']

# "y" pour stratifier y, sinon "None"
stratification = "None"

# Afficher les résultats détaillés (True/False)
details = False

for i in targets:

    y_column=i

    print(f"=========== [{i}] ===========")

    # Split & Scale du dataset
    split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification)

    X_train = X_train.toarray()


    ##################################
    # Création du réseau de neurones #
    ##################################


    def initialize_model():
        

        model = models.Sequential()
    #On indique à notre modèle la dimension des données d'entrées qui correspond au nombre de colonnes de X_train
        model.add(keras.Input(shape=(X_train.shape[1]))) 
    #On met la première couche de notre réseau de neurones. Nous avons une couche avec 140 perceptrons car nous avons
    #140 colonnes dans nos données d'entrée comme dit plus haut. La fonction sigmoid est particulièrement indiquée
    #pour les problèmes de classification. Nous vous encourageons à aller voir à quoi elle ressemble. 
    #    model.add(layers.Dense(86, input_dim=2, activation='sigmoid'))
    #Nous ajoutons une seconde couche car nous sommes dans un problème non linéaire comme dit dans le cours
        model.add(layers.Dense(X_train.shape[1], activation='swish')) #, input_dim=2, activation='sigmoid'
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))
        model.add(layers.Dense(X_train.shape[1], activation='swish'))


        # elu 605
        # relu 609
        # selu 620
        # softplus 603
        # swish 634

        model.add(layers.Dense(1)) #, activation='sigmoid')


        optimizer = Adam(
            learning_rate=0.001,
            decay=0.0,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-09,
            amsgrad=False
        )

        # valeurs par défaut : 
        #'learning_rate': 0.001,
        #'decay': 0.0,
        #'beta_1': 0.9,
        #'beta_2': 0.999,
        #'epsilon': 1e-07,
        # 'amsgrad': False

        
    #Ici, nous pouvons ajouter des paramètres à notre modèle. Il faut juste retenir que "accuracy" permet d'avoir
    #la précision de notre modèle et est particulièrement indiqué pour les problèmes de classification. 
        model.compile(metrics=['mse'],
                    loss='mean_absolute_error',
                    optimizer=optimizer
                    )
        
        return model

    model = initialize_model()

    #"summary" appliqué à notre modèle nous permet d'avoir les paramètres qui la compose ainsi que les dimensions de
    #notre échantillon à la sortie de chaque couche

    #model.summary()

    # Quelques infos supplémentaires sur l'optimiser choisi, notamment le learning_rate
    #model.optimizer.get_config()




    model = initialize_model()

    history = model.fit(X_train, 
                        y_train, 
                        #epochs = 50,
                        #batch_size = 25,
                        verbose = 0)
    
    y_pred = model.predict(X_test, verbose = 0)

    print("--- Entraînement simple : ---")
    print(f"Scores du modèle pour prédire {i} :")
    print(f"R² : {colorama.Style.BRIGHT}{colorama.Back.GREEN}{colorama.Fore.BLACK} {metrics.r2_score(y_test, y_pred):.3f} {colorama.Style.RESET_ALL}")
    print(f"MSE : {metrics.mean_squared_error(y_test, y_pred):.4}")
    print(f"MAE : {metrics.mean_absolute_error(y_test, y_pred):.4}")

    print("--- GridSearch : ---")
    model_grid = KerasRegressor(build_fn=initialize_model, verbose=0)

    # Fixer les valeurs des hyperparamètres à tester
    param_grid = {'epochs':[30, 50], 'batch_size':[10, 25]}

    # Déterminer le score qu'on veut optimiser
    score = 'r2'

    # Je le sors ici car je vais réutiliser cette valeur plus tard
    cv = 5

    grid = model_selection.GridSearchCV(
        model_grid, # On indique le modèle à tester
        param_grid,     # hyperparamètres à tester
        cv=cv,           # nombre de folds de validation croisée
        scoring=score   # score à optimiser
    )

    # Optimiser ce modèle sur le jeu d'entraînement
    grid.fit(X_train, y_train)


    ###########################
    # Affichage des résultats #
    ###########################

    # Afficher le(s) hyperparamètre(s) optimaux
    print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (): {grid.best_params_}")

    y_pred = grid.predict(X_test)
    print(f"Score sur le jeu de test (, avec paramètres optimaux) :")
    print(f"R² : {colorama.Style.BRIGHT}{colorama.Back.GREEN}{colorama.Fore.BLACK} {metrics.r2_score(y_test, y_pred):.3f} {colorama.Style.RESET_ALL}")
    print(f"MSE : {metrics.mean_squared_error(y_test, y_pred):.4}")
    print(f"MAE : {metrics.mean_absolute_error(y_test, y_pred):.4}")
    print('_'*80, "\n")


=========== [SiteEnergyUse_kBtu] ===========
--- Entraînement simple : ---
Scores du modèle pour prédire SiteEnergyUse_kBtu :
R² :  -0.332 
MSE : 2.418e+14
MAE : 7.762e+06
--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (): {'batch_size': 25, 'epochs': 30}
Score sur le jeu de test (, avec paramètres optimaux) :
R² :  0.572 
MSE : 7.765e+13
MAE : 3.679e+06
________________________________________________________________________________ 

=========== [TotalGHGEmissions] ===========
--- Entraînement simple : ---
Scores du modèle pour prédire TotalGHGEmissions :
R² :  -0.040 
MSE : 9.907e+04
MAE : 126.7
--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (): {'batch_size': 10, 'epochs': 50}
Score sur le jeu de test (, avec paramètres optimaux) :
R² :  0.638 
MSE : 3.447e+04
MAE : 90.61
________________________________________________________________________________ 

